# 학습용 데이터 셋 생성 

Color_histogram + Normal_histogram + Lable = training_set.sav 생성을 목표로 함

- [capture_features.py](https://github.com/mkhuthir/RoboND-Perception-Project/blob/master/src/sensor_stick/scripts/capture_features.py)
- [features.py](https://github.com/mkhuthir/RoboND-Perception-Project/blob/master/src/sensor_stick/src/sensor_stick/features.py)

In [1]:
import numpy as np
import pcl

In [33]:
from pcl_helper import *
#https://github.com/mkhuthir/RoboND-Perception-Project/blob/master/src/sensor_stick/scripts/pcl_helper.py

In [3]:
import matplotlib.colors

def rgb_to_hsv(rgb_list):
    rgb_normalized = [1.0*rgb_list[0]/255, 1.0*rgb_list[1]/255, 1.0*rgb_list[2]/255]
    hsv_normalized = matplotlib.colors.rgb_to_hsv([[rgb_normalized]])[0][0]
    return hsv_normalized

def compute_color_histograms_PCD(cloud, using_hsv=False):

    # Compute histograms for the clusters
    point_colors_list = []

    """
    # Step through each point in the point cloud for ROS msg
    for point in pc2.read_points(cloud, skip_nans=True): 
        rgb_list = float_to_rgb(point[3])
        if using_hsv:
            point_colors_list.append(rgb_to_hsv(rgb_list) * 255)
        else:
            point_colors_list.append(rgb_list)
    """
    
    # Step through each point in the point cloud for PCD
    for point in cloud[:,3]: # for PCD file
        rgb_list = float_to_rgb(point)
        if using_hsv:
            point_colors_list.append(rgb_to_hsv(rgb_list) * 255)
        else:
            point_colors_list.append(rgb_list)

    # Populate lists with color values
    channel_1_vals = []
    channel_2_vals = []
    channel_3_vals = []

    for color in point_colors_list:
        channel_1_vals.append(color[0])
        channel_2_vals.append(color[1])
        channel_3_vals.append(color[2])
    
    # Compute histograms
    nbins=32
    bins_range=(0, 256)
        
    # Compute the histogram of the channels separately
    channel_1_hist = np.histogram(channel_1_vals, bins=nbins, range=bins_range)
    channel_2_hist = np.histogram(channel_2_vals, bins=nbins, range=bins_range)
    channel_3_hist = np.histogram(channel_3_vals, bins=nbins, range=bins_range)
    
    # Concatenate the histograms into a single feature vector
    hist_features = np.concatenate((channel_1_hist[0], channel_2_hist[0], channel_1_hist[0])).astype(np.float64)
    
    # Normalize the result
    normed_features = hist_features / np.sum(hist_features)

    # Generate random features for demo mode.  
    # Replace normed_features with your feature vector
    #normed_features = np.random.random(96) 

    # Return the feature vector
    return normed_features 

In [4]:

def get_normals(cloud_path):
    """
    The actual *compute* call from the NormalEstimation class does nothing internally but:
    for each point p in cloud P
        1. get the nearest neighbors of p
        2. compute the surface normal n of p
        3. check if n is consistently oriented towards the viewpoint and flip otherwise

    # normals: pcl._pcl.PointCloud_Normal,size: 26475
    # cloud: pcl._pcl.PointCloud
    """
    cloud = pcl.load(cloud_path)
    
    feature = cloud.make_NormalEstimation()
    #feature.set_RadiusSearch(0.1) #Use all neighbors in a sphere of radius 3cm
    
    feature.set_KSearch(3)
    normals = feature.compute()
    
    return normals

def compute_normal_histograms(normal_cloud, nbins=32, nrange=(-1,1)):
    '''
    Computes and bins the point-cloud data using the objects distribution of surface normals.
    :param: normal_cloud, point cloud containing the filtered clusters.
    :param: nbins,number of bins that data will be pooled into.
    :param: nrange, value range of the data to be pooled.
    :return: the normalised histogram of surface normals
    '''
    norm_x_vals = []
    norm_y_vals = []
    norm_z_vals = []

    for I in range(0,normal_cloud.size):
        norm_x_vals.append(normal_cloud[I][0])
        norm_y_vals.append(normal_cloud[I][1])
        norm_z_vals.append(normal_cloud[I][2])

    # Compute histograms of normal values (just like with color)
    norm_x_hist = np.histogram(norm_x_vals, bins=nbins, range=nrange)
    norm_y_hist = np.histogram(norm_y_vals, bins=nbins, range=nrange)
    norm_z_hist = np.histogram(norm_z_vals, bins=nbins, range=nrange) 

    # Concatenate and normalize the histograms
    hist_features = np.concatenate((norm_x_hist[0], norm_y_hist[0], norm_z_hist[0])).astype(np.float64)
    normed_features = hist_features / np.sum(hist_features)

    return normed_features

In [6]:
cloud_path = "tabletop.pcd"
cloud = pcl.load_XYZRGB(cloud_path)
cloud_arr = cloud.to_array()

In [7]:
# Generate Color Histogram for the spawned model
# Enable using_hsv for better results
c_hists = compute_color_histograms_PCD(cloud_arr, using_hsv=True)

In [8]:
# Generate normals and notmal histograms for the spawned model
normals = get_normals(cloud_path)
n_hists = compute_normal_histograms(normals)

/usr/lib/python2.7/dist-packages/numpy/lib/function_base.py:748: RuntimeWarning: invalid value encountered in greater_equal
  keep = (tmp_a >= mn)
/usr/lib/python2.7/dist-packages/numpy/lib/function_base.py:749: RuntimeWarning: invalid value encountered in less_equal
  keep &= (tmp_a <= mx)


In [9]:
# Generate feature by concatenate of color and normals.
feature = np.concatenate((c_hists, n_hists))

실행 에러 : [소스 코드](https://github.com/mkhuthir/RoboND-Perception-Project/blob/master/src/sensor_stick/scripts/capture_features.py#L52) 확인 필요 

```python 
models = [\
    #'arm_part',
    #'beer',
    'biscuits',
    'book',
    #'bowl',
    #'create',
    #'disk_part',
    'eraser',
    'glue',
    #'hammer',
    #'plastic_cup',
    'snacks',
    'soap',
    'soap2',
    #'soda_can',
    'sticky_notes'
    ]

# add feature to list
labeled_features.append([feature, model_name])
```

In [ ]:
import pickle
#pickle.dump(labeled_features open('training_set.sav', 'wb'))
#https://raw.githubusercontent.com/dexter800/RoboND-Perception-Project/master/training_set.sav

# [train_svm.py](https://github.com/mkhuthir/RoboND-Perception-Project/blob/master/src/sensor_stick/scripts/train_svm.py)
- training_set.sav 활용 model.sav 생성

In [10]:
#!/usr/bin/env python
import pickle
import itertools
import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import cross_validate #from sklearn import cross_validation
from sklearn import metrics

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, '{0:.2f}'.format(cm[i, j]),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
# Load training data from disk
training_set = pickle.load(open('training_set.sav', 'rb'))

# Format the features and labels for use with scikit learn
feature_list = []
label_list = []

for item in training_set:
    if np.isnan(item[0]).sum() < 1:
        feature_list.append(item[0])
        label_list.append(item[1])

print('Features in Training Set: {}'.format(len(training_set)))
print('Invalid Features in Training set: {}'.format(len(training_set)-len(feature_list)))

In [ ]:
X = np.array(feature_list)
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X)
# Apply the scaler to X
X_train = X_scaler.transform(X)
y_train = np.array(label_list)

# Convert label strings to numerical encoding
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)

> sklearn 업데이트로 코드 수정 필요 

In [ ]:
# Create classifier
# Options linear, rbf, sigmoid, poly.
clf = svm.SVC(kernel='linear')

# Set up 5-fold cross-validation
kf = cross_validation.KFold(len(X_train),
                            n_folds=5,
                            shuffle=True,
                            random_state=1)

# Perform cross-validation
scores = cross_validation.cross_val_score(cv=kf,
                                         estimator=clf,
                                         X=X_train, 
                                         y=y_train,
                                         scoring='accuracy'
                                        )
print('Scores: ' + str(scores))
print('Accuracy: %0.2f (+/- %0.2f)' % (scores.mean(), 2*scores.std()))

In [ ]:
# Gather predictions
predictions = cross_validation.cross_val_predict(cv=kf,
                                          estimator=clf,
                                          X=X_train, 
                                          y=y_train
                                         )

accuracy_score = metrics.accuracy_score(y_train, predictions)
print('accuracy score: '+str(accuracy_score))

confusion_matrix = metrics.confusion_matrix(y_train, predictions)

class_names = encoder.classes_.tolist()


#Train the classifier
clf.fit(X=X_train, y=y_train)

model = {'classifier': clf, 'classes': encoder.classes_, 'scaler': X_scaler}

# Save classifier to disk
pickle.dump(model, open('model.sav', 'wb'))

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(confusion_matrix, classes=encoder.classes_,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(confusion_matrix, classes=encoder.classes_, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

# [예측 ](https://github.com/mkhuthir/RoboND-Perception-Project/blob/master/src/sensor_stick/scripts/object_recognition.py#L142)

In [34]:
model = pickle.load(open('model.sav', 'rb'))
#https://raw.githubusercontent.com/dexter800/RoboND-Perception-Project/master/model.sav

In [35]:
clf = model['classifier']
encoder = LabelEncoder()
encoder.classes_ = model['classes']
scaler = model['scaler']

In [36]:
cloud_path = "table_scene_inliers_0.pcd"
cloud = pcl.load_XYZRGB(cloud_path)
cloud_arr = cloud.to_array()

In [37]:
# Extract histogram features
################################
# Generate Color hist
c_hists = compute_color_histograms_PCD(cloud_arr, using_hsv=True)

# Generate normals and notmal histograms for the spawned model
################################
normals = get_normals(cloud_path)
n_hists = compute_normal_histograms(normals)

# Generate feature by concatenate of color and normals.
################################
feature = np.concatenate((c_hists, n_hists))

In [38]:
detected_objects = []
# Make the prediction, retrieve the label for the result
# and add it to detected_objects_labels list
################################
prediction = clf.predict(scaler.transform(feature.reshape(1,-1)))
label = encoder.inverse_transform(prediction)[0]

In [39]:
label

'soap2'